<a href="https://colab.research.google.com/github/pradytpk/ml-notes/blob/main/ML73_Random_Forest_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RandomForest Classifier Implementation With Pipelines And Hyperparameter Tuning

In [28]:
import seaborn as sns
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [29]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

## Process Needed to be followed
1. Handling Missing Values
2. Handling Categorical features
3. Handling outliers
4. Feature scaling
5. Automating the entire

In [30]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [32]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [33]:
df['time'].value_counts()

0    176
1     68
Name: time, dtype: int64

In [34]:
## independent and dependent features
X=df.drop(labels=['time'],axis=1)
y=df['time']

In [35]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [36]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [37]:
X_train.head()

,total_bill,tip,sex,smoker,day,size
228,13.28,2.72,Male,No,Sat,2
208,24.27,2.03,Male,Yes,Sat,2
96,27.28,4.00,Male,Yes,Fri,2
167,31.71,4.50,Male,No,Sun,4
84,15.98,2.03,Male,No,Thur,2


In [38]:
from sklearn.impute import SimpleImputer ## Handling Missing Values
from sklearn.preprocessing import OneHotEncoder## handling Categorical features
from sklearn.preprocessing import StandardScaler## Feature scaling
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
## Automating the entire

In [39]:
categorical_cols = ['sex', 'smoker','day']
numerical_cols = ['total_bill', 'tip','size']

In [40]:
## Feature Engineering Automation
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')), ##missing values
        ('scaler',StandardScaler())## feature scaling
        ]
)

#categorical Pipeline
cat_pipeline=Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='most_frequent')), ## handling Missing values
                ('onehotencoder',OneHotEncoder()) ## Categorical features to numerical
                ]
            )

In [41]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)

])

In [42]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [44]:
## Automate Model Training Process
models={
    'Random Forest':RandomForestClassifier(),
    'Decision Tree':DecisionTreeClassifier(),
    'SVC':SVC()
}

In [45]:
from sklearn.metrics import accuracy_score

In [46]:
def evaluate_model(X_train,y_train,X_test,y_test,models):
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        # Train model
        model.fit(X_train,y_train)
        # Predict Testing data
        y_test_pred =model.predict(X_test)
        # Get accuracy for test data prediction
        test_model_score = accuracy_score(y_test,y_test_pred)
        report[list(models.keys())[i]] =  test_model_score
    return report

In [47]:
evaluate_model(X_train,y_train,X_test,y_test,models)

{'Random Forest': 0.9591836734693877,
 'Decision Tree': 0.9387755102040817,
 'SVC': 0.9591836734693877}

In [48]:
classifier=RandomForestClassifier()

In [49]:
## Hypeparameter Tuning
params={'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['gini','entropy']
              }

In [50]:
from sklearn.model_selection import RandomizedSearchCV

In [51]:
cv=RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.974 total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.923 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.974 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.923 total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.923 total time=   0.1s
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=0.974 total time=   0.3s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=0.949 total time=   0.3s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=0.974 total time=   0.3s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=0.923 total time=   0.3s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=0.949 total time=   0.3s
[CV 

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [52]:
cv.best_params_

{'n_estimators': 200, 'max_depth': None, 'criterion': 'gini'}